In [40]:
import folium
import pandas as pd
import geopandas as gpd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

In [41]:
m = folium.Map(location=[-34.679725714625725, -58.45085487390574], zoom_start =14)

In [42]:
m.save("primermapa.html")

In [43]:
tooltip = "Haz click"
folium.Marker([-34.679725714625725, -58.45085487390574], popup="ETEC", tooltip=tooltip, icon= folium.Icon(color="red",icon="info-sign")).add_to(m)
m.save("primermapa.html")

In [44]:
folium.Circle(
    radius = 250,
    location = [-34.679725714625725, -58.45085487390574],
    popup = "Zona de medicion",
    color = "#3186cc",
    fill = "#3186cc"
).add_to(m)
m.save("primermapa.html")

In [45]:
df = pd.read_csv("../data/mediciones.csv")


In [46]:
df["fecha_hora"] = pd.to_datetime(df["fecha_hora"])

In [47]:
df

,id,zona,latitud,longitud,temperatura,humedad,co2,fecha_hora
0,1,ETEC,-34.679726,-58.450855,25.4,60.2,410,2025-11-11 17:16:06
1,2,ETEC,-34.679726,-58.450855,27.1,63.2,620,2025-11-11 17:17:03
2,3,ETEC,-34.679726,-58.450855,27.1,63.2,620,2025-11-11 17:17:50
3,4,ETEC,-34.679726,-58.450855,27.5,63.5,670,2025-11-11 17:18:03
4,5,ETEC,-34.679726,-58.450855,27.5,63.5,670,2025-11-11 17:18:16
5,6,ETEC,-34.679726,-58.450855,28.5,64.0,680,2025-11-11 17:18:29
6,7,ETEC,-34.679726,-58.450855,28.5,64.0,680,2025-11-11 17:18:42
7,8,ETEC,-34.679726,-58.450855,28.5,64.0,680,2025-11-11 17:18:54
8,9,ETEC,-34.679726,-58.450855,30.7,67.9,691,2025-11-11 17:19:06
9,10,ETEC,-34.679726,-58.450855,30.7,67.9,691,2025-11-11 17:19:19


In [48]:
output_dir = "resultados"
os.makedirs(output_dir, exist_ok=True)

In [49]:
zonas = df["zona"].unique()
zona_widget = widgets.Dropdown(options=zonas, description="Zona:")
fecha_inicio_widget = widgets.DatePicker(description="Desde:", value=df["fecha_hora"].min().date())
fecha_fin_widget = widgets.DatePicker(description="Hasta:", value=df["fecha_hora"].max().date())
boton_actualizar = widgets.Button(description="Guardar", button_style='info')

In [50]:
def actualizar(_):
    clear_output(wait=True)
    display(zona_widget, fecha_inicio_widget, fecha_fin_widget, boton_actualizar)

    zona = zona_widget.value
    f_ini = fecha_inicio_widget.value
    f_fin = fecha_fin_widget.value

    df_f = df[
        (df["zona"] == zona)
        & (df["fecha_hora"].dt.date >= f_ini)
        & (df["fecha_hora"].dt.date <= f_fin)
    ]

    if df_f.empty:
        print("⚠️ No hay datos en el rango seleccionado.")
        return

    temp_max = df_f["temperatura"].max()
    temp_min = df_f["temperatura"].min()
    temp_mean = df_f["temperatura"].mean()
    co2_mean = df_f["co2"].mean()

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(df_f["fecha_hora"], df_f["co2"], label="CO₂ (ppm)", marker="o")
    ax.plot(df_f["fecha_hora"], df_f["temperatura"], label="Temperatura (°C)", marker="s")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Valores")
    ax.legend()
    ax.grid(True)

    grafico_path = os.path.join(output_dir, f"grafico_{zona}.png")
    plt.savefig(grafico_path, bbox_inches="tight")
    plt.close(fig)

    # Guardar todo en un archivo .txt
    archivo_path = os.path.join(output_dir, f"resumen_{zona}.txt")
    with open(archivo_path, "w", encoding="utf-8") as f:
        f.write(f"Zona: {zona}\n")
        f.write(f"Rango de fechas: {f_ini} a {f_fin}\n\n")
        f.write("📊 Estadísticas:\n")
        f.write(f"Temperatura máxima: {temp_max:.2f} °C\n")
        f.write(f"Temperatura promedio: {temp_mean:.2f} °C\n")
        f.write(f"Temperatura mínima: {temp_min:.2f} °C\n")
        f.write(f"CO₂ promedio: {co2_mean:.2f} ppm\n\n")
        f.write(f"Gráfico guardado en: {grafico_path}\n\n")
        f.write("📄 Datos filtrados:\n")
        f.write(df_f.to_string(index=False))

    print(f"✅ Archivo generado: {archivo_path}")
    print(f"🖼️ Gráfico: {grafico_path}")

In [51]:
boton_actualizar.on_click(actualizar)


In [52]:
display(zona_widget, fecha_inicio_widget, fecha_fin_widget, boton_actualizar)

Dropdown(description='Zona:', options=('ETEC',), value='ETEC')

DatePicker(value=datetime.date(2025, 11, 11), description='Desde:', step=1)

DatePicker(value=datetime.date(2025, 11, 11), description='Hasta:', step=1)

Button(button_style='info', description='Guardar', style=ButtonStyle())

In [54]:
df.to_csv("resultados.txt", sep="\t", index=False)


In [56]:
df

,id,zona,latitud,longitud,temperatura,humedad,co2,fecha_hora
0,1,ETEC,-34.679726,-58.450855,25.4,60.2,410,2025-11-11 17:16:06
1,2,ETEC,-34.679726,-58.450855,27.1,63.2,620,2025-11-11 17:17:03
2,3,ETEC,-34.679726,-58.450855,27.1,63.2,620,2025-11-11 17:17:50
3,4,ETEC,-34.679726,-58.450855,27.5,63.5,670,2025-11-11 17:18:03
4,5,ETEC,-34.679726,-58.450855,27.5,63.5,670,2025-11-11 17:18:16
5,6,ETEC,-34.679726,-58.450855,28.5,64.0,680,2025-11-11 17:18:29
6,7,ETEC,-34.679726,-58.450855,28.5,64.0,680,2025-11-11 17:18:42
7,8,ETEC,-34.679726,-58.450855,28.5,64.0,680,2025-11-11 17:18:54
8,9,ETEC,-34.679726,-58.450855,30.7,67.9,691,2025-11-11 17:19:06
9,10,ETEC,-34.679726,-58.450855,30.7,67.9,691,2025-11-11 17:19:19
